In [0]:
# %r
#  /FileStore/tables/clientes.json
#  /FileStore/tables/testando/clientes.json

In [0]:
%fs rm -r /user/hive/warehouse/compras

res1: Boolean = false

In [0]:

import pyspark


In [0]:
dfcliente = (spark.read.format("json").load("/FileStore/tables/testando/clientes.json"))
display(dfcliente)

customer,date_order,id,price,product,unit
Carlos,2021-01-23,1,238.0,TV,5
Ana,2021-03-22,2,121.6,AR-CONDICIONADO,6
Sofia,2021-04-21,3,415.4,FREEZER,7
Sandra,2021-04-23,4,313.0,TV,8
Tereza,2021-05-23,5,412.0,FRIGOBAR,3
Carla,2021-06-25,6,124.0,MESA,1
Sofia,2021-07-25,7,342.3,CADEIRA,1


In [0]:
dfcliente.printSchema()

root
 |-- customer: string (nullable = true)
 |-- date_order: string (nullable = true)
 |-- id: long (nullable = true)
 |-- price: double (nullable = true)
 |-- product: string (nullable = true)
 |-- unit: long (nullable = true)



In [0]:
dfcliente.createOrReplaceTempView("compras_cliente")

saida = spark.sql("SELECT * FROM compras_cliente")
saida.show()

+--------+----------+---+-----+---------------+----+
|customer|date_order| id|price|        product|unit|
+--------+----------+---+-----+---------------+----+
|  Carlos|2021-01-23|  1|238.0|             TV|   5|
|     Ana|2021-03-22|  2|121.6|AR-CONDICIONADO|   6|
|   Sofia|2021-04-21|  3|415.4|        FREEZER|   7|
|  Sandra|2021-04-23|  4|313.0|             TV|   8|
|  Tereza|2021-05-23|  5|412.0|       FRIGOBAR|   3|
|   Carla|2021-06-25|  6|124.0|           MESA|   1|
|   Sofia|2021-07-25|  7|342.3|        CADEIRA|   1|
+--------+----------+---+-----+---------------+----+



In [0]:
%scala

val scrisql = "CREATE OR REPLACE TABLE compras(id string, date_order string, customer string, price float,product string, unit integer) USING DELTA PARTITIONED BY (date_order)"
spark.sql(scrisql)

scrisql: String = CREATE OR REPLACE TABLE compras(id string, date_order string, customer string, price float,product string, unit integer) USING DELTA PARTITIONED BY (date_order)
res2: org.apache.spark.sql.DataFrame = []

In [0]:
spark.sql("SELECT * FROM compras").show()

+---+----------+--------+-----+-------+----+
| id|date_order|customer|price|product|unit|
+---+----------+--------+-----+-------+----+
+---+----------+--------+-----+-------+----+



In [0]:
%scala

val mergedados = "Merge into compras " +
                  "using compras_cliente as cp_view " +
                  "ON compras.id = cp_view.id " +
                  "WHEN MATCHED THEN " +
                  "UPDATE SET compras.product = cp_view.product, " +
                  "compras.price = cp_view.price " +
                  "WHEN NOT MATCHED THEN INSERT * "
spark.sql(mergedados).show()

+-----------------+----------------+----------------+-----------------+
num_affected_rows|num_updated_rows|num_deleted_rows|num_inserted_rows|
+-----------------+----------------+----------------+-----------------+
 7| 0| 0| 7|
+-----------------+----------------+----------------+-----------------+

mergedados: String = "Merge into compras using compras_cliente as cp_view ON compras.id = cp_view.id WHEN MATCHED THEN UPDATE SET compras.product = cp_view.product, compras.price = cp_view.price WHEN NOT MATCHED THEN INSERT * "

In [0]:
spark.sql("SELECT * FROM compras").show()

+---+----------+--------+-----+---------------+----+
| id|date_order|customer|price|        product|unit|
+---+----------+--------+-----+---------------+----+
|  2|2021-03-22|     Ana|121.6|AR-CONDICIONADO|   6|
|  5|2021-05-23|  Tereza|412.0|       FRIGOBAR|   3|
|  7|2021-07-25|   Sofia|342.3|        CADEIRA|   1|
|  3|2021-04-21|   Sofia|415.4|        FREEZER|   7|
|  6|2021-06-25|   Carla|124.0|           MESA|   1|
|  4|2021-04-23|  Sandra|313.0|             TV|   8|
|  1|2021-01-23|  Carlos|238.0|             TV|   5|
+---+----------+--------+-----+---------------+----+



In [0]:
%scala

val at_dados =  "update compras set product = 'geladeira' where id = 4"
spark.sql(at_dados)

at_dados: String = update compras set product = 'geladeira' where id = 4
res5: org.apache.spark.sql.DataFrame = [num_affected_rows: bigint]

In [0]:
spark.sql("SELECT * FROM compras").show()

+---+----------+--------+-----+---------------+----+
| id|date_order|customer|price|        product|unit|
+---+----------+--------+-----+---------------+----+
|  2|2021-03-22|     Ana|121.6|AR-CONDICIONADO|   6|
|  4|2021-04-23|  Sandra|313.0|      geladeira|   8|
|  5|2021-05-23|  Tereza|412.0|       FRIGOBAR|   3|
|  7|2021-07-25|   Sofia|342.3|        CADEIRA|   1|
|  3|2021-04-21|   Sofia|415.4|        FREEZER|   7|
|  6|2021-06-25|   Carla|124.0|           MESA|   1|
|  1|2021-01-23|  Carlos|238.0|             TV|   5|
+---+----------+--------+-----+---------------+----+



In [0]:
%scala

val delete = "delete from compras where id = 1"
spark.sql(delete)

delete: String = delete from compras where id = 1
res7: org.apache.spark.sql.DataFrame = [num_affected_rows: bigint]

In [0]:
spark.sql("SELECT * FROM compras").show()

+---+----------+--------+-----+---------------+----+
| id|date_order|customer|price|        product|unit|
+---+----------+--------+-----+---------------+----+
|  2|2021-03-22|     Ana|121.6|AR-CONDICIONADO|   6|
|  4|2021-04-23|  Sandra|313.0|      geladeira|   8|
|  5|2021-05-23|  Tereza|412.0|       FRIGOBAR|   3|
|  7|2021-07-25|   Sofia|342.3|        CADEIRA|   1|
|  3|2021-04-21|   Sofia|415.4|        FREEZER|   7|
|  6|2021-06-25|   Carla|124.0|           MESA|   1|
+---+----------+--------+-----+---------------+----+



In [0]:
%sql 
DESCRIBE HISTORY 'user/hive/warehouse/compras'

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
3,2024-02-21T20:43:38.000+0000,7406534468247714,anacadriano20@gmail.com,DELETE,"Map(predicate -> [""(cast(id#2882 as int) = 1)""])",null,List(2275985682911563),0221-202852-us9uczep,2,WriteSerializable,false,"Map(numRemovedFiles -> 1, numRemovedBytes -> 1499, numCopiedRows -> 0, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 4166, numDeletedRows -> 1, scanTimeMs -> 2483, numAddedFiles -> 0, numAddedBytes -> 0, rewriteTimeMs -> 1683)",null,Databricks-Runtime/12.2.x-scala2.12
2,2024-02-21T20:43:22.000+0000,7406534468247714,anacadriano20@gmail.com,UPDATE,"Map(predicate -> [""(cast(id#2097 as int) = 4)""])",null,List(2275985682911563),0221-202852-us9uczep,1,WriteSerializable,false,"Map(numRemovedFiles -> 1, numRemovedBytes -> 1499, numCopiedRows -> 0, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 6771, scanTimeMs -> 3488, numAddedFiles -> 1, numUpdatedRows -> 1, numAddedBytes -> 1549, rewriteTimeMs -> 3268)",null,Databricks-Runtime/12.2.x-scala2.12
1,2024-02-21T20:42:55.000+0000,7406534468247714,anacadriano20@gmail.com,MERGE,"Map(predicate -> [""(cast(id#1158 as bigint) = id#513L)""], matchedPredicates -> [{""actionType"":""update""}], notMatchedPredicates -> [{""actionType"":""insert""}], notMatchedBySourcePredicates -> [])",null,List(2275985682911563),0221-202852-us9uczep,0,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 7, numTargetBytesAdded -> 10673, numTargetBytesRemoved -> 0, numTargetDeletionVectorsAdded -> 0, numTargetRowsMatchedUpdated -> 0, executionTimeMs -> 12340, numTargetRowsInserted -> 7, numTargetRowsMatchedDeleted -> 0, scanTimeMs -> 3268, numTargetRowsUpdated -> 0, numOutputRows -> 7, numTargetDeletionVectorsRemoved -> 0, numTargetRowsNotMatchedBySourceUpdated -> 0, numTargetChangeFilesAdded -> 0, numSourceRows -> 7, numTargetFilesRemoved -> 0, numTargetRowsNotMatchedBySourceDeleted -> 0, rewriteTimeMs -> 8136)",null,Databricks-Runtime/12.2.x-scala2.12
0,2024-02-21T20:42:27.000+0000,7406534468247714,anacadriano20@gmail.com,CREATE OR REPLACE TABLE,"Map(isManaged -> true, description -> null, partitionBy -> [""date_order""], properties -> {})",null,List(2275985682911563),0221-202852-us9uczep,null,WriteSerializable,true,Map(),null,Databricks-Runtime/12.2.x-scala2.12


In [0]:
%sql 
DESCRIBE DETAIL 'user/hive/warehouse/compras'

format,id,name,description,location,createdAt,lastModified,partitionColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures,statistics
delta,657a39bf-2fa0-45e1-8928-da5254165e5a,null,null,dbfs:/user/hive/warehouse/compras,2024-02-21T20:42:25.099+0000,2024-02-21T20:43:38.000+0000,List(date_order),6,9224,Map(),1,2,"List(appendOnly, invariants)",Map()


In [0]:
%sql
SELECT * FROM delta. `/user/hive/warehouse/compras` VERSION AS OF 10


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-2275985682911579>:7
      5     display(df)
      6     return df
----> 7   _sqldf = ____databricks_percent_sql()
      8 finally:
      9   del ____databricks_percent_sql

File <command-2275985682911579>:4, in ____databricks_percent_sql()
      2 def ____databricks_percent_sql():
      3   import base64
----> 4   df = spark.sql(base64.standard_b64decode("U0VMRUNUICogRlJPTSBkZWx0YS4gYC91c2VyL2hpdmUvd2FyZWhvdXNlL2NvbXByYXNgIFZFUlNJT04gQVMgT0YgMTA=").decode())
      5   display(df)
      6   return df

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, sig

In [0]:
%sql
CREATE TABLE delta. `/Temporario/hive/clonado` CLONE delta. `/user/hive/warehouse/compras`

In [0]:
%sql
RESTORE TABLE compras TO VERSION AS OF 3

In [0]:
spark.sql("select * from compras").show()

In [0]:
spark.sql("update compras set product = 'GELADEIRA' where id = 4")
spark.sql("select * from compras").show()